In [2]:
import os
import h5py
import math
import time

import torch 
import torch.nn as nn
import torchvision.models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import models.resnet as resnet
import collections
import cv2
import numpy as np
import dataset
import PIL.Image as Image
from IPython.display import clear_output

In [4]:
checkpoint=torch.load("/home/olaf/DLC/CV_LTH_Pre-training/Detection/weight/imagenet_weight.pt")

model=torchvision.models.resnet50(weights='IMAGENET1K_V1') #is hetzelfde als onze weight file
model.train()

num_features = model.fc.in_features
print(model.layer4)
upsample_block = nn.Sequential(
    nn.Upsample(scale_factor=2),
)
model.fc= nn.Upsample(scale_factor=2) # hier moet je dus aanpassen maar de input shape is 2048, data collecter werkt voor mij lokaal wel
# print(model.fc)
print(model)

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): 

In [4]:

def h5_loader(path):
    h5f = h5py.File(path, "r")
    rgb = np.array(h5f['rgb'])
    rgb = np.transpose(rgb, (1, 2, 0))
    
    rgb=Image.fromarray(rgb)
    # 
    depth = Image.fromarray(np.array(h5f['depth']))
    return rgb, depth

def log10(x):
    """Convert a new tensor with the base-10 logarithm of the elements of x. """
    return torch.log(x) / math.log(10)

class Result(object):
    def __init__(self):
        self.irmse, self.imae = 0, 0
        self.mse, self.rmse, self.mae = 0, 0, 0
        self.absrel, self.lg10 = 0, 0
        self.delta1, self.delta2, self.delta3 = 0, 0, 0
        self.data_time, self.gpu_time = 0, 0

    def set_to_worst(self):
        self.irmse, self.imae = np.inf, np.inf
        self.mse, self.rmse, self.mae = np.inf, np.inf, np.inf
        self.absrel, self.lg10 = np.inf, np.inf
        self.delta1, self.delta2, self.delta3 = 0, 0, 0
        self.data_time, self.gpu_time = 0, 0

    def update(self, irmse, imae, mse, rmse, mae, absrel, lg10, delta1, delta2, delta3, gpu_time, data_time):
        self.irmse, self.imae = irmse, imae
        self.mse, self.rmse, self.mae = mse, rmse, mae
        self.absrel, self.lg10 = absrel, lg10
        self.delta1, self.delta2, self.delta3 = delta1, delta2, delta3
        self.data_time, self.gpu_time = data_time, gpu_time

    def evaluate(self, output, target):
        valid_mask = target > 0
        output = output[valid_mask]
        target = target[valid_mask]

        abs_diff = (output - target).abs()

        self.mse = float((torch.pow(abs_diff, 2)).mean())
        self.rmse = math.sqrt(self.mse)
        self.mae = float(abs_diff.mean())
        self.lg10 = float((log10(output) - log10(target)).abs().mean())
        self.absrel = float((abs_diff / target).mean())

        maxRatio = torch.max(output / target, target / output)
        self.delta1 = float((maxRatio < 1.25).float().mean())
        self.delta2 = float((maxRatio < 1.25 ** 2).float().mean())
        self.delta3 = float((maxRatio < 1.25 ** 3).float().mean())
        self.data_time = 0
        self.gpu_time = 0

        inv_output = 1 / output
        inv_target = 1 / target
        abs_inv_diff = (inv_output - inv_target).abs()
        self.irmse = math.sqrt((torch.pow(abs_inv_diff, 2)).mean())
        self.imae = float(abs_inv_diff.mean())

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
valdir="/home/olaf/DLC/dataset/val/images/"

the_list=os.listdir(valdir)
transform=transforms.Compose([
            transforms.Resize(400),
            transforms.CenterCrop((300,400)),
            transforms.ToTensor(),
            
        ])
# transform(h5_loader(valdir+the_list[0])[0])
optimizer=torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.01,weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
batchsize=3

In [6]:
def train_transform(rgb, depth):
        s = np.random.uniform(1.0, 1.5)  # random scaling
        depth_np = depth 
        

        # perform 1st step of data augmentation
        # transform = transforms.Compose([
        #     transforms.Resize(256),  # this is for computational efficiency, since rotation can be slow
        #     # transforms.Resize(s),
        #     transforms.CenterCrop((228)),
        #     # transforms.HorizontalFlip(do_flip)
        # ])
        rgb_np = rgb#transform(rgb)
        # rgb_np = color_jitter(rgb_np)  # random color jittering
        rgb_np = np.asfarray(rgb_np, dtype='float') / 255
        depth_np = np.asfarray( depth_np ,dtype='float')

        return rgb_np, depth_np

In [7]:
valdir="/home/olaf/DLC/dataset/train/images/"
the_list=os.listdir(valdir)
the_list=the_list[:120]

In [17]:
model.train()
avg=0
rho=0.01
end = time.time()
for i,img_path in enumerate(the_list[::batchsize]):
    
    im_list=torch.Tensor()
    depth_list=torch.Tensor()
    i=batchsize*i
    for id in range(i,i+batchsize):
        try:
            
            im,depth=h5_loader(valdir+the_list[id])
        except:
            print('failed')
            pass
        else:
            im=transform(im)
            # depth=transforms.ToTensor()(depth)
            # depth=normalize(depth)
            depth=transform(depth)
            im_list=torch.cat((im_list,im))
            depth_list=torch.cat((depth_list,depth))
    
    im_tensor=im_list.reshape((batchsize,*im.shape))
    depth_list=depth_list.reshape((batchsize,*depth.shape))
    im_tensor=torch.Tensor(np.array(im_tensor))#.transpose(1,3)
    print(im_tensor.shape)
    target=torch.Tensor(np.array(depth_list))

    data_time = time.time() - end
    end =time.time()
    output=model(im_tensor)
    print(output.shape)
    # print(im_tensor.shape,depth_tensor.shape,output.shape)
    loss = criterion(output,target)
    avg=(avg*(1-rho)+loss*rho)/(1-rho**(i+0.5))
    print(loss,avg)
    

    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

    gpu_time = time.time() - end

    # measure accuracy and record loss
    result = Result()
    result.evaluate(output.data, target.data)
    # average_meter.update(result, gpu_time, data_time, input.size(0))
    end = time.time()
    clear_output()
    cv2.imshow('image',output[0].detach().numpy())
    cv2.imshow('original',target[0].numpy())
    cv2.imshow('ori image',im_tensor[0].transpose(0,2).numpy())
    cv2.waitKey(1)
    # print(im_tensor.shape)


    print(i)

torch.Size([3, 3, 300, 400])


NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 2D) for the modes: nearest | linear | bilinear | bicubic | trilinear | area | nearest-exact (got nearest)